In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import time

In [49]:
import torch

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
image = cv2.imread("training/down.0.jpg")

In [4]:
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, static_image_mode=False, max_num_hands=1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [61]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # changes from bgr to rgb since cv2 is bgr but mediapipe requires rgb
image.flags.writeable = False
results = hands.process(image) # this makes the actual detections
image.flags.writeable = True
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

In [71]:
landmarks = []
if results.multi_hand_landmarks:
    for landmark in results.multi_hand_landmarks[0].landmark:
        x, y, z = landmark.x, landmark.y, landmark.z
        landmarks.append([x,y,z])

In [74]:
landmarks = np.array(landmarks)

In [76]:
landmarks_tensor = torch.tensor(landmarks)

In [78]:
landmarks_tensor.shape

torch.Size([21, 3])

In [83]:
landmarks_tensor

tensor([[ 2.6169e-01,  3.5162e-01,  5.2388e-07],
        [ 3.1173e-01,  4.0588e-01, -1.9637e-02],
        [ 3.4277e-01,  5.1171e-01, -4.9626e-02],
        [ 3.4353e-01,  6.2118e-01, -7.7466e-02],
        [ 3.4383e-01,  6.8985e-01, -1.0540e-01],
        [ 3.6291e-01,  4.3615e-01, -8.1996e-02],
        [ 3.5962e-01,  5.8429e-01, -1.1503e-01],
        [ 3.5429e-01,  6.6624e-01, -1.3190e-01],
        [ 3.4618e-01,  7.2988e-01, -1.4240e-01],
        [ 3.2038e-01,  4.1795e-01, -9.0673e-02],
        [ 3.1555e-01,  5.9374e-01, -1.2266e-01],
        [ 3.1004e-01,  5.6808e-01, -1.1885e-01],
        [ 3.1328e-01,  5.2203e-01, -1.1607e-01],
        [ 2.7819e-01,  4.1597e-01, -9.8668e-02],
        [ 2.8027e-01,  5.8395e-01, -1.1922e-01],
        [ 2.7786e-01,  5.5161e-01, -9.8715e-02],
        [ 2.8025e-01,  5.0518e-01, -8.4991e-02],
        [ 2.4063e-01,  4.2561e-01, -1.0939e-01],
        [ 2.4922e-01,  5.6303e-01, -1.1770e-01],
        [ 2.5351e-01,  5.4196e-01, -1.0227e-01],
        [ 2.5585e-01

In [127]:
data = []
y_vals = []
for i in range(10):
    image = cv2.imread(f"training/down.{i}.jpg")

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # changes from bgr to rgb since cv2 is bgr but mediapipe requires rgb
    image.flags.writeable = False
    results = hands.process(image) # this makes the actual detections
    
    landmarks = []
    if results.multi_hand_landmarks:
        for landmark in results.multi_hand_landmarks[0].landmark:
            x, y = landmark.x, landmark.y
            landmarks.append([x,y])
        data.append(landmarks)
        y_vals.append(0)

for i in range(10):
    image = cv2.imread(f"training/up.{i}.jpg")

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # changes from bgr to rgb since cv2 is bgr but mediapipe requires rgb
    image.flags.writeable = False
    results = hands.process(image) # this makes the actual detections
    
    landmarks = []
    if results.multi_hand_landmarks:
        for landmark in results.multi_hand_landmarks[0].landmark:
            x, y = landmark.x, landmark.y
            landmarks.append([x,y])
        data.append(landmarks)
        y_vals.append(1)



In [128]:
y = torch.tensor(y_vals)

In [129]:
y.shape

torch.Size([16])

In [130]:
data = torch.tensor(data)

In [133]:
data.shape

torch.Size([16, 21, 2])

In [152]:
m = torch.nn.Flatten(0)

In [151]:
data[0].shape

torch.Size([21, 2])

In [153]:
m(data[0]).shape

torch.Size([42])

In [134]:
import torch.nn as nn
import torch.functional as F

In [170]:
class HandNetwork(nn.Module):
    def __init__(self):
        super(HandNetwork, self).__init__()
        self.flatten = nn.Flatten(0)
        self.fc1 = nn.Linear(42, 100)
        self.relu = nn.ReLU(True)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 2)
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [171]:
model = HandNetwork()
loss_fn = torch.nn.BCELoss()

In [172]:
output = model(data[0])

In [174]:
output

tensor([0.0601, 0.1512], grad_fn=<ViewBackward0>)